In [1]:
#Inverted index
#* Input: a textual file containing a set of sentences
#* Each line of the file has the following format:  sentenceId\tsentence\n
#* Output: report for each word w the list of sentenceIds of the sentences containing w
#* Do not consider the words “and”, “or”, “not”

In [2]:
import findspark
findspark.init()
findspark.find()

'C:\\Users\\Study\\anaconda3\\lib\\site-packages\\pyspark'

In [3]:
import os
cwd = os.getcwd()
print(cwd)

C:\Users\Study\Documents\Jupyter Notebooks\PoliTO\1-30 MapReduce Exercises\Exercise-7


In [4]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Exercise 7") \
    .getOrCreate()
sc = spark.sparkContext

In [5]:
spark

In [6]:
sc

<SparkContext master=local[*] appName=Exercise 7>

In [7]:
rdd7_raw = sc.textFile('input_file_7.txt')
rdd7_raw.take(10)

['Sentence#1\tHadoop or Spark',
 'Sentence#2\tHadoop or Spark and Java',
 'Sentence#3\tHadoop and Big Data']

In [8]:
rdd7_raw.first()

'Sentence#1\tHadoop or Spark'

In [9]:
rdd7_processing = rdd7_raw.map(lambda _row: _row.split('\t') ).map(lambda __row: (__row[0],__row[1])  )
rdd7_processing.first()[1]

'Hadoop or Spark'

In [10]:
rdd7_processing.take(10)


[('Sentence#1', 'Hadoop or Spark'),
 ('Sentence#2', 'Hadoop or Spark and Java'),
 ('Sentence#3', 'Hadoop and Big Data')]

In [11]:
rdd7_cleaning = rdd7_processing.map(lambda row: (row[0],row[1].replace('and','')))\
                               .map(lambda row: (row[0],row[1].replace('or','')))\
                               .map(lambda row: (row[0],row[1].replace('  ',' ')))
#rdd7_cleaning = rdd7_cleaning.map(lambda row: row[1].replace('or','' ))


In [12]:
rdd7_cleaning.take(10)

[('Sentence#1', 'Hadoop Spark'),
 ('Sentence#2', 'Hadoop Spark Java'),
 ('Sentence#3', 'Hadoop Big Data')]

In [13]:
rdd7_distribution = rdd7_cleaning.map(lambda row: (row[0],row[1].split(' '))) 
rdd7_distribution.take(10)

[('Sentence#1', ['Hadoop', 'Spark']),
 ('Sentence#2', ['Hadoop', 'Spark', 'Java']),
 ('Sentence#3', ['Hadoop', 'Big', 'Data'])]

In [14]:
def distribute(tp):
    total_list = []
    print(tp)
    for elem in tp[1]:
        current_tuple = (elem, tp[0])
        total_list.append(current_tuple)
    return(total_list)


In [15]:
rdd7_distribution = rdd7_distribution.map(distribute)
rdd7_distribution.take(10)

[[('Hadoop', 'Sentence#1'), ('Spark', 'Sentence#1')],
 [('Hadoop', 'Sentence#2'), ('Spark', 'Sentence#2'), ('Java', 'Sentence#2')],
 [('Hadoop', 'Sentence#3'), ('Big', 'Sentence#3'), ('Data', 'Sentence#3')]]

In [16]:
rdd7_distribution = rdd7_distribution.flatMap(lambda x:x)
rdd7_distribution.take(10)

[('Hadoop', 'Sentence#1'),
 ('Spark', 'Sentence#1'),
 ('Hadoop', 'Sentence#2'),
 ('Spark', 'Sentence#2'),
 ('Java', 'Sentence#2'),
 ('Hadoop', 'Sentence#3'),
 ('Big', 'Sentence#3'),
 ('Data', 'Sentence#3')]

In [25]:
rdd7_result = rdd7_distribution.reduceByKey(lambda a,b : a+','+b ) 
rdd7_result= rdd7_result.map(lambda x: (x[0],x[1].split(',')) )
rdd7_result.take(10)

[('Spark', ['Sentence#1', 'Sentence#2']),
 ('Java', ['Sentence#2']),
 ('Hadoop', ['Sentence#1', 'Sentence#2', 'Sentence#3']),
 ('Big', ['Sentence#3']),
 ('Data', ['Sentence#3'])]

In [18]:
rdd7_distribution.take(10)

[('Hadoop', 'Sentence#1'),
 ('Spark', 'Sentence#1'),
 ('Hadoop', 'Sentence#2'),
 ('Spark', 'Sentence#2'),
 ('Java', 'Sentence#2'),
 ('Hadoop', 'Sentence#3'),
 ('Big', 'Sentence#3'),
 ('Data', 'Sentence#3')]

In [19]:
def to_list(a):
    return [a]

def append(a, b):
    a.append(b)
    return a

def extend(a, b):
    a.extend(b)
    return a

rdd7_result2 = rdd7_distribution.combineByKey(to_list,append,extend )
rdd7_result2.take(10)

[('Spark', ['Sentence#1', 'Sentence#2']),
 ('Java', ['Sentence#2']),
 ('Hadoop', ['Sentence#1', 'Sentence#2', 'Sentence#3']),
 ('Big', ['Sentence#3']),
 ('Data', ['Sentence#3'])]

In [20]:
rdd7_result3 = rdd7_distribution.combineByKey(lambda a: [a], lambda a,b: a.append(b), lambda a,b: a.extend(b) )
rdd7_result3.take(10)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 27.0 failed 1 times, most recent failure: Lost task 0.0 in stage 27.0 (TID 27) (DESKTOP-8UHQ6HB executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\Users\Study\anaconda3\lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 604, in main
  File "C:\Users\Study\anaconda3\lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 594, in process
  File "C:\Users\Study\anaconda3\lib\site-packages\pyspark\rdd.py", line 2916, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "C:\Users\Study\anaconda3\lib\site-packages\pyspark\rdd.py", line 418, in func
    return f(iterator)
  File "C:\Users\Study\anaconda3\lib\site-packages\pyspark\rdd.py", line 2152, in _mergeCombiners
    merger.mergeCombiners(ite

In [ ]:
print(type([]))

In [ ]:
print('list' if type([])=='list' else 'not list')